In [1]:
import pandas as pd
import plotly.express as px
from copy import copy
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from tensorflow import keras
import pandas_datareader as pdr

In [22]:
#tickers = ['JPM']
stock_df = pdr.DataReader('JPM', 'yahoo')
stock_df.reset_index(drop=False, inplace=True)
stock_df

,Date,High,Low,Open,Close,Volume,Adj Close
0,2016-05-27,9.930000,9.500000,9.880000,9.630000,2404700,9.630000
1,2016-05-31,9.730000,9.460000,9.700000,9.530000,3815700,9.530000
2,2016-06-01,9.650000,9.350000,9.500000,9.600000,2171700,9.600000
3,2016-06-02,9.680000,9.400000,9.500000,9.650000,2332700,9.650000
4,2016-06-03,9.580000,9.330000,9.570000,9.460000,3083000,9.460000
...,...,...,...,...,...,...,...
1253,2021-05-20,206.690002,202.199997,202.589996,204.910004,8336500,204.910004
1254,2021-05-21,208.899994,199.619995,207.000000,200.009995,9230000,200.009995
1255,2021-05-24,212.800003,197.300003,202.169998,210.949997,13215000,210.949997
1256,2021-05-25,219.089996,211.770004,212.070007,216.199997,14779000,216.199997


In [23]:
training_data = stock_df.iloc[:,2:3]
training_data

,Low
0,9.500000
1,9.460000
2,9.350000
3,9.400000
4,9.330000
...,...
1253,202.199997
1254,199.619995
1255,197.300003
1256,211.770004


In [24]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_data)
training_set_scaled

array([[0.00407286],
       [0.00392201],
       [0.00350719],
       ...,
       [0.71229779],
       [0.76686657],
       [0.78206436]])

In [25]:
x = []
y = []

for i in range (1, len(training_data)):
    x.append(training_set_scaled[i-1:i,0])#actual data
    y.append(training_set_scaled[i,0]) #what we want to be predicted

In [26]:
x = np.asarray(x)
y = np.asarray(y)

In [27]:
split = int(0.7*len(x))
x_train = x[:split]
x_test = x[split:]
y_train = y[:split]
y_test = y[split:]

In [28]:
#convert to a 3D array tp run model
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1)) #(x,y,z)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [29]:
inputs = keras.layers.Input(shape=(x_train.shape[1],x_train.shape[2]))
x=keras.layers.LSTM(150, return_sequences=True)(inputs)
x=keras.layers.LSTM(150, return_sequences=True)(x)
x=keras.layers.LSTM(150, return_sequences=True)(x)
x=keras.layers.LSTM(150, return_sequences=True)(x)
outputs = keras.layers.Dense(1,activation='linear')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss="mse")
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1, 1)]            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 1, 150)            91200     
_________________________________________________________________
lstm_5 (LSTM)                (None, 1, 150)            180600    
_________________________________________________________________
lstm_6 (LSTM)                (None, 1, 150)            180600    
_________________________________________________________________
lstm_7 (LSTM)                (None, 1, 150)            180600    
_________________________________________________________________
dense_1 (Dense)              (None, 1, 1)              151       
Total params: 633,151
Trainable params: 633,151
Non-trainable params: 0
________________________________________________

In [30]:
history  = model.fit(x_train, y_train, epochs=50, batch_size = 32, validation_split = 0.2)

Epoch 1/50
22/22 [==============================] - 3s 150ms/step - loss: 0.0120 - val_loss: 0.0071
Epoch 2/50
22/22 [==============================] - 0s 11ms/step - loss: 0.0080 - val_loss: 0.0130
Epoch 3/50
22/22 [==============================] - 0s 12ms/step - loss: 0.0062 - val_loss: 0.0072
Epoch 4/50
22/22 [==============================] - 0s 12ms/step - loss: 6.3407e-04 - val_loss: 2.3252e-04
Epoch 5/50
22/22 [==============================] - 0s 13ms/step - loss: 1.1711e-04 - val_loss: 5.5310e-05
Epoch 6/50
22/22 [==============================] - 0s 13ms/step - loss: 4.4731e-05 - val_loss: 4.9001e-05
Epoch 7/50
22/22 [==============================] - 0s 14ms/step - loss: 4.0131e-05 - val_loss: 4.3865e-05
Epoch 8/50
22/22 [==============================] - 0s 13ms/step - loss: 3.6762e-05 - val_loss: 4.3372e-05
Epoch 9/50
22/22 [==============================] - 0s 16ms/step - loss: 3.6473e-05 - val_loss: 4.3521e-05
Epoch 10/50
22/22 [==============================] - 0s 17ms

In [31]:
#recreating the shape for x
x = []
for i in range (1, len(training_data)):
    x.append(training_set_scaled[i-1:i,0])#actual data
x = np.asarray(x)

predictions = model.predict(x)

In [32]:
predictions.shape
test_predictions = []

for i in predictions:
    test_predictions.append(i[0])
    
test_predictions = sc.inverse_transform(test_predictions)
test_predictions

array([[  8.98021452],
       [  8.93979255],
       [  8.82862676],
       ...,
       [188.00317889],
       [186.06838113],
       [198.00612471]])

In [35]:
df_predicted = stock_df[1:][['Date','Close']]
df_predicted['Prediction'] = test_predictions
df_predicted

,Date,Close,Prediction
1,2016-05-31,9.530000,8.980215
2,2016-06-01,9.600000,8.939793
3,2016-06-02,9.650000,8.828627
4,2016-06-03,9.460000,8.879157
5,2016-06-06,9.510000,8.808413
...,...,...,...
1253,2021-05-20,204.910004,181.796049
1254,2021-05-21,200.009995,190.145548
1255,2021-05-24,210.949997,188.003179
1256,2021-05-25,216.199997,186.068381


In [36]:
def interactive_plot(df):
    fig = px.line()
    
    for i in df.columns[1:]:
        fig.add_scatter(x=df['Date'], y =df[i], name = i)
        
    fig.show()
interactive_plot(df_predicted)   